In [1]:
### Here, we will be loading and manipulating the core data that will be used for the remainder of the python
### notebooks. Some of the variables defined here are
###    -estimated inversion strength (EIS), 
###    -sea surface temperatures (SSTs), 
###    -horizontal temp advection (Tadv), 
###    -relative humidity at 700 hPa (RH700), 
###    -pressure vertical velocity at 700 hPa (w700), and
###    -near surface wind speed (WS)
###    -TOA fluxes

In [2]:
##### LOAD FUNCTIONS NOTEBOOK AND LIBRARIES #####

In [3]:
# Import packages here
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy
from cartopy import config
import cartopy.crs as ccrs

import climlab
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import warnings

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
warnings.filterwarnings('ignore')
%matplotlib inline

/data/keeling/a/tjhanke2/miniconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [4]:
%run ./get_data_variables_fncs.ipynb
%run ./get_global_constants.ipynb

In [5]:
##### GET CCF VARIABLES #####

In [5]:
## Load the ERA5 Data
path = '/data/keeling/a/tjhanke2/enso-energy-budget/Data/raw_data/ERA5_data/'
met_params = xr.open_mfdataset(path+'*.nc').load()

In [6]:
## Expanded time domain
T700 = met_params.t           # temp data at 700 hPa (K)
T2m = met_params.t2m          # temp data at 2 m (K)
SST = met_params.sst          # temp data at sea surface (K)
U10 = met_params.u10.interp(latitude=new_lat_medRes,longitude=new_lon_medRes)  # horizontal vel at 10 m (m/s)
V10 = met_params.v10.interp(latitude=new_lat_medRes,longitude=new_lon_medRes)  # meridional vel at 10 m (m/s)
ws = met_params.si10          # wind vel at 10 m (m/s)
RH700 = met_params.r          # rel humidity at 700 hPa (%)
w700 = met_params.w * 864     # vert vel at 700 hPa (hPa/day)

## Get EIS and Tadv
EIS = climlab.thermo.EIS(T2m,T700)

dSST_dlat = SST.interp(latitude=new_lat_medRes,longitude=new_lon_medRes).differentiate(coord='latitude',edge_order=2)
dSST_dlon = SST.interp(latitude=new_lat_medRes,longitude=new_lon_medRes).differentiate(coord='longitude',edge_order=2)
Tadv = ((-U10/(R_E*weight) * dSST_dlon) - (V10/R_E * dSST_dlat)) * 86400 * 180/np.pi

In [4]:
## Slice the data to focus on the CERES interval (2002 - 2019)
T700 = met_params.t.sel(time=slice('2002-07','2019'))           # temp data at 700 hPa (K)
T2m = met_params.t2m.sel(time=slice('2002-07','2019'))          # temp data at 2 m (K)
SST = met_params.sst.sel(time=slice('2002-07','2019'))          # temp data at sea surface (K)
U10 = met_params.u10.sel(time=slice('2002-07','2019')).interp(latitude=new_lat_medRes,longitude=new_lon_medRes)  # horizontal vel at 10 m (m/s)
V10 = met_params.v10.sel(time=slice('2002-07','2019')).interp(latitude=new_lat_medRes,longitude=new_lon_medRes)  # meridional vel at 10 m (m/s)
ws = met_params.si10.sel(time=slice('2002-07','2019'))          # wind vel at 10 m (m/s)
RH700 = met_params.r.sel(time=slice('2002-07','2019'))          # rel humidity at 700 hPa (%)
w700 = met_params.w.sel(time=slice('2002-07','2019')) * 864     # vert vel at 700 hPa (hPa/day)

## Get EIS and Tadv
EIS = climlab.thermo.EIS(T2m,T700)

dSST_dlat = SST.interp(latitude=new_lat_medRes,longitude=new_lon_medRes).differentiate(coord='latitude',edge_order=2)
dSST_dlon = SST.interp(latitude=new_lat_medRes,longitude=new_lon_medRes).differentiate(coord='longitude',edge_order=2)
Tadv = ((-U10/(R_E*weight) * dSST_dlon) - (V10/R_E * dSST_dlat)) * 86400 * 180/np.pi

In [8]:
##### GET CERES DATASETS #####

In [9]:
## Load CERES-FBCT
path = '/data/keeling/a/tjhanke2/enso-energy-budget/Data/raw_data/CERES_data/'
FBCT_flux = xr.merge([xr.open_mfdataset(path+'*.nc').sel(time=slice('2002-07','2019-12')),
                      xr.open_mfdataset(path+'cloud_props/*.nc').sel(time=slice('2002-07','2019-12'))
                    ]).load()
FBCT_flux = FBCT_flux.rename({'lat':'latitude','lon':'longitude'})
FBCT_flux['time'] = SST.time

In [10]:
## Load CERES-EBAF
path = '/data/keeling/a/tjhanke2/enso-energy-budget/Data/raw_data/CERES_data/ebaf/'
EBAF_flux = xr.open_mfdataset(path+'*.nc').sel(time=slice('2002-07','2019-12')).load()
EBAF_flux = EBAF_flux.rename({'lat':'latitude','lon':'longitude'})
EBAF_flux['time'] = SST.time

In [14]:
##### GET RADIATIVE KERNELS #####

In [11]:
# Load CCF kernel from Ryan Scott's github
kernel = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/raw_data/CRE_kernel_data/Scott_Myers_meteorological_kernels_CERES-FBCT_ERA5.nc')
clr_kernel = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/raw_data/CLR_SKY_kernel_data/TOA_CloudSat_Kerns.nc', decode_times=False).interp(latitude=new_lat_medRes,longitude=new_lon_medRes).rename({'time':'month'})
plevels = clr_kernel.plev.values

# Shift lon from [-180,180] to [0,360]
kernel_rolled = kernel.assign_coords(longitude=((kernel.longitude % 360) - 2.5)).roll(longitude=(kernel.dims['longitude'] // 2)).roll(longitude=180).sortby('longitude')

In [16]:
##### BEGIN GETTING CALCULATED VARIABLES #####

In [7]:
## Get anomalies and low resolution versions
SST_anom = get_anomaly(SST,1)
EIS_anom = get_anomaly(EIS,1)
Tadv_anom = get_anomaly(Tadv,1)
RH700_anom = get_anomaly(RH700,1)
w700_anom = get_anomaly(w700,1)
ws_anom = get_anomaly(ws,1)

# Downsample the CCFs
SST_anom_medRes = SST_anom.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
EIS_anom_medRes = EIS_anom.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
Tadv_anom_medRes = Tadv_anom.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
RH700_anom_medRes = RH700_anom.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
w700_anom_medRes = w700_anom.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
ws_anom_medRes = ws_anom.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)

SST_anom_loRes = SST_anom.interp(latitude=new_lat_loRes, longitude=new_lon_loRes)
EIS_anom_loRes = EIS_anom.interp(latitude=new_lat_loRes, longitude=new_lon_loRes)
Tadv_anom_loRes = Tadv_anom.interp(latitude=new_lat_loRes, longitude=new_lon_loRes)
RH700_anom_loRes = RH700_anom.interp(latitude=new_lat_loRes, longitude=new_lon_loRes)
w700_anom_loRes = w700_anom.interp(latitude=new_lat_loRes, longitude=new_lon_loRes)
ws_anom_loRes = ws_anom.interp(latitude=new_lat_loRes, longitude=new_lon_loRes)

# Store CCFs into dataset
ccf_anom_loRes = xr.Dataset(data_vars={'SST_anom':SST_anom_loRes,
                                       'EIS_anom':EIS_anom_loRes,
                                       'Tadv_anom':Tadv_anom_loRes,
                                       'RH700_anom':RH700_anom_loRes,
                                       'w700_anom':w700_anom_loRes,
                                       'ws_anom':ws_anom_loRes},
                            coords={'latitude':new_lat_loRes,
                                    'longitude':new_lon_loRes,
                                    'time':SST_anom_loRes.time})

ccf_anom_medRes = xr.Dataset(data_vars={'SST_anom':SST_anom_medRes,
                                       'EIS_anom':EIS_anom_medRes,
                                       'Tadv_anom':Tadv_anom_medRes,
                                       'RH700_anom':RH700_anom_medRes,
                                       'w700_anom':w700_anom_medRes,
                                       'ws_anom':ws_anom_medRes},
                            coords={'latitude':new_lat_medRes,
                                    'longitude':new_lon_medRes,
                                    'time':SST_anom_medRes.time})


ccf_anom_hiRes = xr.Dataset(data_vars={'SST_anom':SST_anom,
                                       'EIS_anom':EIS_anom,
                                       'Tadv_anom':Tadv_anom,
                                       'RH700_anom':RH700_anom,
                                       'w700_anom':w700_anom,
                                       'ws_anom':ws_anom},
                            coords={'latitude':SST_anom.latitude,
                                    'longitude':SST_anom.longitude,
                                    'time':SST_anom.time})

In [24]:
ccf_anom_medRes = xr.Dataset(data_vars={'SST_anom':SST_anom_medRes,
                                       'EIS_anom':EIS_anom_medRes,
                                       'Tadv_anom':Tadv_anom_medRes,
                                       'RH700_anom':RH700_anom_medRes,
                                       'w700_anom':w700_anom_medRes,
                                       'ws_anom':ws_anom_medRes},
                            coords={'latitude':new_lat_medRes,
                                    'longitude':new_lon_medRes,
                                    'time':SST_anom_medRes.time})

In [25]:
## Save low resolution anomalies
#ccf_anom_loRes.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/expanded_domain/ccf_anom_lo_res_1979.nc', mode='w')

## Save med resolution anomalies
ccf_anom_medRes.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/expanded_domain/ccf_anom_med_res_1979.nc', mode='w')

## Save high resolution anomalies
#ccf_anom_hiRes.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/expanded_domain/ccf_anom_hi_res_1979.nc', mode='w')

In [23]:
## Get nino3.4 region anomalies
nino34 = SST_anom.sel(latitude=slice(5,-5),longitude=slice(190,240))
nino34_mean = nino34.weighted(weight).mean(dim=('longitude','latitude'))
nino34_rolling = nino34_mean.rolling(time=3,center=True).mean()

## Get the nino3.4 composite indexer
nino34 = xr.full_like(nino34_rolling,'none',dtype='U4')
nino34[nino34_rolling >= 0.5] = 'nino'
nino34[nino34_rolling <= -0.5] = 'nina'

## Save nino3.4 index
nino34_rolling.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/expanded_domain/nino34_rolling_1979.nc')
nino34.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/expanded_domain/nino34_index_1979.nc')

In [14]:
## Get CERES radiation anomalies
loCld_cre_anom = calc_fbct_cre(FBCT_flux.toa_lw_cldtyp_mon.sel(press=slice(0.0,1.0)),FBCT_flux.toa_lw_clr_mon,
                               FBCT_flux.toa_sw_cldtyp_mon.sel(press=slice(0.0,1.0)),FBCT_flux.toa_sw_clr_mon,
                               FBCT_flux.cldarea_cldtyp_mon.sel(press=slice(0.0,1.0)) / 100)

highCld_cre_anom = calc_fbct_cre(FBCT_flux.toa_lw_cldtyp_mon.sel(press=slice(2.0,6.0)),FBCT_flux.toa_lw_clr_mon,
                                 FBCT_flux.toa_sw_cldtyp_mon.sel(press=slice(2.0,6.0)),FBCT_flux.toa_sw_clr_mon,
                                 FBCT_flux.cldarea_cldtyp_mon.sel(press=slice(2.0,6.0)) / 100)

net_cre_anom = calc_net_cre()

clr_sky_anom = calc_ebaf_rad_anom(EBAF_flux.toa_net_clr_t_mon)

all_sky_anom = calc_ebaf_rad_anom(EBAF_flux.toa_net_all_mon)

# Store radiation into dataset
ceres_anom_loRes = xr.Dataset(data_vars={'loCld_cre':loCld_cre_anom,
                                         'highCld_cre':highCld_cre_anom,
                                         'net_cre':net_cre_anom,
                                         'clr_sky':clr_sky_anom,
                                         'all_sky':all_sky_anom},
                            coords={'latitude':new_lat_loRes,
                                    'longitude':new_lon_loRes,
                                    'time':all_sky_anom.time})

## Save low resolution anomalies
#ceres_anom_loRes.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ceres_anom/ceres_anom_med_res.nc')

In [7]:
## Get delta specific humidity (q) and delta T (atmos. temp.)
# Load temperature and relative humidity data
temp_data = xr.open_dataset('/data/keeling/a/cristi/a/data/ERA5/ERA5_mon_1979-2019_temp.nc').t.sel(level=plevels,time=slice('2002-07','2019')).rename({'level':'plev'})
rh_data = xr.open_dataset('/data/keeling/a/cristi/a/data/ERA5/ERA5_mon_1979-2019_rh.nc').r.sel(level=plevels,time=slice('2002-07','2019')).rename({'level':'plev'})
skin_temp_data = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/raw_data/ERA5_data/ERA5_mon_1979-2019_skin_temp.nc').skt.sel(expver=1,time=slice('2002-07','2019'))
del skin_temp_data['expver']

# Get low resolution versions
temp_data_loRes = temp_data.interp(latitude=new_lat_medRes,longitude=new_lon_medRes)
skin_temp_data_loRes = skin_temp_data.interp(latitude=new_lat_medRes,longitude=new_lon_medRes)
rh_data_loRes = rh_data.interp(latitude=new_lat_medRes,longitude=new_lon_medRes)

# Use Clausius-Clapeyron and other met. relationships to get specific humidity
e_sat = climlab.thermo.clausius_clapeyron(temp_data_loRes)
e_sat_clim = climlab.thermo.clausius_clapeyron(temp_data_loRes.mean(dim='time'))
e_sat_clim_1 = climlab.thermo.clausius_clapeyron(temp_data_loRes.mean(dim='time')+1)

e = (rh_data_loRes/100) * e_sat
e_clim = (rh_data_loRes.mean(dim='time')/100) * e_sat_clim
e_clim_1 = (rh_data_loRes.mean(dim='time')/100) * e_sat_clim_1

w = (e*287) / (461*(clr_kernel.plev-e))
w_clim = (e_clim*287) / (461*(clr_kernel.plev-e_clim))
w_clim_1 = (e_clim_1*287) / (461*(clr_kernel.plev-e_clim_1))

q = w / (w+1)
q_clim = w_clim / (w_clim+1)
q_clim_1 = w_clim_1 / (w_clim_1+1)

# Finally, get delta t,q
del_atmt = get_anomaly(temp_data_loRes,1)
del_skt = get_anomaly(skin_temp_data_loRes,1)
del_t = xr.concat([del_skt.assign_coords({'plev':1013}),del_atmt],dim='plev')
del_q = (np.log(q) - np.log(q_clim)) / (np.log(q_clim_1) - np.log(q_clim))

In [8]:
## Get CLR SKY contributions to radiative anomalies using kernel decomposition
# Radiation anomaly contributed by atmos. & skin temp.
clr_kernel_t = xr.concat([clr_kernel.lwclr_ts.assign_coords({'plev':1013}),clr_kernel.lwclr_t],dim='plev')
dR_anom_by_dT = get_mean_across_lev(clr_kernel_t,del_t)

# Radiation anomaly contributed by specific humidity
dR_anom_by_dq_lw = get_mean_across_lev(clr_kernel.lwclr_q,del_q)
dR_anom_by_dq_sw = get_mean_across_lev(clr_kernel.swclr_q,del_q)

# Radiation anomaly contributed by variable humidity
dR_anom_by_dq_lw_fix = get_mean_across_lev(clr_kernel.lwclr_q,del_t.sel(plev=slice(1000,0)))
dR_anom_by_dq_lw_var = dR_anom_by_dq_lw - dR_anom_by_dq_lw_fix

# # Store radiation anomalies into dataset
clr_sky_kernel_decomp = xr.Dataset(data_vars={'dR_anom_by_dT':dR_anom_by_dT,
                                              'dR_anom_by_dq_lw':dR_anom_by_dq_lw,
                                              'dR_anom_by_dq_sw':dR_anom_by_dq_sw,
                                              'dR_anom_by_fixed_q':dR_anom_by_dq_lw_fix},
                                   coords={'latitude':new_lat_medRes,
                                           'longitude':new_lon_medRes,
                                           'time':dR_anom_by_dT.time})

# ## Save CCF CRE contributions
# clr_sky_kernel_decomp.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/clr_sky_kernel_decomp/clr_sky_kernel_medRes_data.nc')

In [15]:
## Get CCF contributions to low cloud CRE
# Separate kernels into dR sensitivities
dR_dSST   = kernel_rolled.dRdxi.sel(i=1.0) # dR/dSST
dR_dEIS   = kernel_rolled.dRdxi.sel(i=2.0) # dR/dEIS
dR_dTadv  = kernel_rolled.dRdxi.sel(i=3.0) # dR/dTadv
dR_dRH700 = kernel_rolled.dRdxi.sel(i=4.0) # dR/dRH700
dR_dw700  = kernel_rolled.dRdxi.sel(i=5.0) # dR/dw700
dR_dws    = kernel_rolled.dRdxi.sel(i=6.0) # dR/dws

# Normalize CCF anomalies, then get CRE contribution
SST_anom_norm = SST_anom_loRes / kernel_rolled.sigma_xi.sel(i=1.0)
SST_CRE = SST_anom_norm * dR_dSST
del SST_CRE['i']

EIS_anom_norm = EIS_anom_loRes / kernel_rolled.sigma_xi.sel(i=2.0)
EIS_CRE = EIS_anom_norm * dR_dEIS
del EIS_CRE['i']

Tadv_anom_norm = Tadv_anom_loRes / kernel_rolled.sigma_xi.sel(i=3.0)
Tadv_CRE = Tadv_anom_norm * dR_dTadv
del Tadv_CRE['i']

RH700_anom_norm = RH700_anom_loRes / kernel_rolled.sigma_xi.sel(i=4.0)
RH700_CRE = RH700_anom_norm * dR_dRH700
del RH700_CRE['i']

w700_anom_norm = w700_anom_loRes / kernel_rolled.sigma_xi.sel(i=5.0)
w700_CRE = w700_anom_norm * dR_dw700
del w700_CRE['i']

ws_anom_norm = ws_anom_loRes / kernel_rolled.sigma_xi.sel(i=6.0)
ws_CRE = ws_anom_norm * dR_dws
del ws_CRE['i']

# Store CCFs into dataset
ccf_cre_anom = xr.Dataset(data_vars={'SST_CRE':SST_CRE,
                                     'EIS_CRE':EIS_CRE,
                                     'Tadv_CRE':Tadv_CRE,
                                     'RH700_CRE':RH700_CRE,
                                     'w700_CRE':w700_CRE,
                                     'ws_CRE':ws_CRE},
                            coords={'latitude':new_lat_loRes,
                                    'longitude':new_lon_loRes,
                                    'time':SST_CRE.time})

## Save CCF CRE contributions
# ccf_cre_anom.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_cre_anom/ccf_cre_anom.nc')

In [14]:
## Load CERES radiation anomaly dataset
ceres_rad_anom = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ceres_anom/ceres_anom_med_res.nc')

loCld_cre_anom = ceres_rad_anom.loCld_cre
highCld_cre_anom = ceres_rad_anom.highCld_cre
net_cre_anom = ceres_rad_anom.net_cre
clr_sky_anom = ceres_rad_anom.clr_sky
all_sky_anom = ceres_rad_anom.all_sky

## Load CCF anomaly dataset
ccf_anom_medRes = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/ccf_anom_med_res.nc')

SST_anom_medRes = ccf_anom_medRes.SST_anom
EIS_anom_medRes = ccf_anom_medRes.EIS_anom
Tadv_anom_medRes = ccf_anom_medRes.Tadv_anom

## Load CCF CRE radiation anomaly dataset
ccf_cre_anom = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_cre_anom/ccf_cre_anom.nc')

SST_CRE = ccf_cre_anom.SST_CRE
EIS_CRE = ccf_cre_anom.EIS_CRE
Tadv_CRE = ccf_cre_anom.Tadv_CRE

## Load CLR SKY kernel decomp dataset
clr_sky_kernel_decomp = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/clr_sky_kernel_decomp/clr_sky_kernel_medRes_data.nc')

dR_anom_by_dT = clr_sky_kernel_decomp.dR_anom_by_dT.transpose('time','latitude','longitude')
dR_anom_by_dq_lw = clr_sky_kernel_decomp.dR_anom_by_dq_lw.transpose('time','latitude','longitude')
dR_anom_by_dq_sw = clr_sky_kernel_decomp.dR_anom_by_dq_sw.transpose('time','latitude','longitude')

nino34_rolling = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/nino3_4/nino34_rolling.nc').__xarray_dataarray_variable__

In [16]:
## Transform variables from time dimension to lag dimension
%run /data/keeling/a/tjhanke2/enso-energy-budget/Code/eof_analysis/get_lagged_regress.ipynb
lag_list = range(-12,13)

# CCFs
SST_anom_lag = time2lag(nino34_rolling,SST_anom_medRes,lag_list)
EIS_anom_lag = time2lag(nino34_rolling,EIS_anom_medRes,lag_list)
Tadv_anom_lag = time2lag(nino34_rolling,Tadv_anom_medRes,lag_list)

# # CERES
all_sky_lag = time2lag(nino34_rolling,all_sky_anom,lag_list)
clr_sky_lag = time2lag(nino34_rolling,clr_sky_anom,lag_list)
net_cre_lag = time2lag(nino34_rolling,net_cre_anom,lag_list)
loCld_cre_lag = time2lag(nino34_rolling,loCld_cre_anom,lag_list)
highCld_cre_lag = time2lag(nino34_rolling,highCld_cre_anom,lag_list)

# # CCF CRE
SST_CRE_lag = time2lag(nino34_rolling,SST_CRE,lag_list)
EIS_CRE_lag = time2lag(nino34_rolling,EIS_CRE,lag_list)
Tadv_CRE_lag = time2lag(nino34_rolling,Tadv_CRE,lag_list)

# CLR SKY KERNELS
# dR_dT_lag = time2lag(nino34_rolling,dR_anom_by_dT,lag_list)
# dR_dq_lw_lag = time2lag(nino34_rolling,dR_anom_by_dq_lw,lag_list)
# dR_dq_sw_lag = time2lag(nino34_rolling,dR_anom_by_dq_sw,lag_list)

# Store lagged variables into datasets
ccf_lags = xr.Dataset(data_vars={'SST_anom_lag':SST_anom_lag,
                                 'EIS_anom_lag':EIS_anom_lag,
                                 'Tadv_anom_lag':Tadv_anom_lag},
                            coords={'latitude':new_lat_medRes,
                                    'longitude':new_lon_medRes,
                                    'lag':SST_anom_lag.lag})

ceres_lags = xr.Dataset(data_vars={'all_sky_lag':all_sky_lag,
                                   'clr_sky_lag':clr_sky_lag,
                                   'net_cre_lag':net_cre_lag,
                                   'loCld_cre_lag':loCld_cre_lag,
                                   'highCld_cre_lag':highCld_cre_lag},
                            coords={'latitude':new_lat_medRes,
                                    'longitude':new_lon_medRes,
                                    'lag':all_sky_lag.lag})

ccf_cre_lags = xr.Dataset(data_vars={'SST_CRE_lag':SST_CRE_lag,
                                     'EIS_CRE_lag':EIS_CRE_lag,
                                     'Tadv_CRE_lag':Tadv_CRE_lag},
                            coords={'latitude':new_lat_loRes,
                                    'longitude':new_lon_loRes,
                                    'lag':SST_CRE_lag.lag})

# clr_sky_lags = xr.Dataset(data_vars={'dR_dT_lag':dR_dT_lag,
#                                      'dR_dq_lw_lag':dR_dq_lw_lag,
#                                      'dR_dq_sw_lag':dR_dq_sw_lag},
#                             coords={'latitude':new_lat_medRes,
#                                     'longitude':new_lon_medRes,
#                                     'lag':dR_dT_lag.lag})

## Save CCF CRE contributions
# ccf_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms/ccf_lags.nc')
# ceres_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms/ceres_lags.nc')
# ccf_cre_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms/ccf_cre_lags.nc')
# clr_sky_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms/clr_sky_med_res_lags.nc')

In [ ]:
ccf_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms_detrend/ccf_lags.nc')
ceres_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms_detrend/ceres_lags.nc')
ccf_cre_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms_detrend/ccf_cre_lags_new.nc')
# clr_sky_lags.to_netcdf(path='/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/lagged_anoms_detrend/clr_sky_med_res_lags.nc')